In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = os.getcwd()
root_dir = os.path.abspath(os.path.join(cwd, os.pardir))
root_dir

In [ ]:
def read_outfile(name, nboulders=0, coord=False):
    full_name = os.path.join(root_dir, name)
    if not os.path.isfile(full_name):
        print("El archivo %s no existe"%os.path.basename(full_name))
        return
    if coord:
        df = pd.read_csv(full_name, delimiter='\\s+', header=None,
                         names=["idx",
                                "btype",
                                "t",
                                "theta", "omega", 
                                "x", "y",
                                "vx", "vy",
                                "mass", "radius"])
        df.name = name.split(".")[0]
        df.nt = df.idx.nunique()
        df.nb = nboulders
        df.np = df.nt - nboulders
        df["r"] = np.sqrt(df["x"]**2 + df["y"]**2)
        df["v"] = np.sqrt(df["vx"]**2 + df["vy"]**2)
    else:
        df = pd.read_csv(full_name, delimiter='\\s+', header=None,
                         names=["idx",
                                "btype",
                                "t",
                                "theta", "omega", 
                                "a", "e", "M", "w", "mmr",
                                "mass", "radius",
                                "dist"]) # Asteroid
        df.name = name.split(".")[0]
        df.tmax = df.t.iloc[-1]
    return df

In [ ]:
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection


def circles_with_radius(ax, x, y, radii, **kwargs):
    """
    Plot many circles with radius in *data units* efficiently using PatchCollection.
    """
    patches = [Circle((cx, cy), r) for cx, cy, r in zip(x, y, radii)]
    collection = PatchCollection(patches, **kwargs)
    ax.add_collection(collection)
    return collection

In [ ]:
coord = True
data = read_outfile("salida.out", coord=coord)
data

In [ ]:
this = data.idx == -1
if coord:
    plt.plot(data.x.loc[this], data.y.loc[this], ".--")
    plt.axis("equal")

In [ ]:
ntot = data.idx.nunique()
idsorted = sorted(data.idx.unique())
plt.figure(dpi=150)
ax = plt.gca()
if coord:
    # data[["x", "y", "vx", "vy"]] = data[["x", "y", "vx", "vy"]].values - np.repeat(data[["x", "y", "vx", "vy"]].values[::ntot], ntot, axis=0)
    
    for i in range(ntot):
        this = data.idx == idsorted[i]
        if (data.btype.loc[this].iloc[0] > 1) and (data.btype.loc[this].iloc[0] < 2) and (data.radius.loc[this].iloc[0] > 0.0):
            # Initial condition
            circ = Circle((data.x.loc[this].iloc[0], data.y.loc[this].iloc[0]), data.radius.loc[this].iloc[0],
                           transform=ax.transData, label=str(idsorted[i]), facecolor=f"C{i}", edgecolor=f"C{i}")
            ax.add_patch(circ)
            circles_with_radius(ax, data.x.loc[this], data.y.loc[this], 
                                data.radius.loc[this],
                                facecolor="none", edgecolor=f"C{i}")
        else:
            plt.plot(data.x.loc[this], data.y.loc[this], ".", label=idsorted[i], color=f"C{i}")
    plt.axis("equal")
else:
    for i in range(ntot):
        this = data.idx == idsorted[i]
        # plt.plot(data.t.loc[this], data.a.loc[this], ".", label=idsorted[i])
        # plt.plot(data.a.loc[this], data.e.loc[this], ".", label=idsorted[i])
        plt.plot(data.t.loc[this], data.mmr.loc[this], ".", label=idsorted[i])
    plt.semilogx()
plt.legend()
plt.show()

In [ ]:
for i in range(ntot):
    # plt.plot(data.t, data.theta / np.pi *180, ".")
    plt.plot(data.t, data.omega, ".")
plt.semilogx()